In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import torch
import torchvision
from torchvision import datasets
from torchvision import transforms as T # for simplifying the transforms
from torchvision.transforms import ToTensor

from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader, sampler, random_split
from torchvision import models
import timm
from timm.loss import LabelSmoothingCrossEntropy 
import matplotlib.pyplot as plt
%matplotlib inline
import sys
from tqdm import tqdm
import time
import copy

from linformer import Linformer
from torch.optim.lr_scheduler import StepLR
from vit_pytorch.efficient import ViT

In [2]:
#loading the dataset folder for train and validate
dataset_path = 'M:/VieApp/biophilic_training_dataset'

In [3]:
def get_classes(data_dir):
    all_data = datasets.ImageFolder(data_dir)
    return all_data.classes

In [4]:
def get_data_loaders(data_dir, batch_size, train = False):
    if train:
        #train
        transform = T.Compose([
            T.Resize((256, 256)),
            T.ToTensor(),
            T.Normalize(timm.data.IMAGENET_DEFAULT_MEAN, timm.data.IMAGENET_DEFAULT_STD), # imagenet means
        ])
        train_data = datasets.ImageFolder(os.path.join(data_dir, "train/"), transform = transform)
        print(train_data)
        train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=4)
        return train_loader, len(train_data)
    else:
        # val/test
        transform = T.Compose([ # We dont need augmentation for test transforms
            T.Resize((256, 256)),
            T.ToTensor(),
            T.Normalize(timm.data.IMAGENET_DEFAULT_MEAN, timm.data.IMAGENET_DEFAULT_STD), # imagenet means
        ])
        val_data = datasets.ImageFolder(os.path.join(data_dir, "val/"), transform=transform)
        print(val_data)
        test_data = datasets.ImageFolder(os.path.join(data_dir, "val/"), transform=transform)
        print(test_data)
        val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=True, num_workers=1)
        test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True, num_workers=1)
        return val_loader, test_loader, len(val_data), len(test_data)

In [5]:
classes = get_classes("M:/VieApp/biophilic_training_dataset/train/")
print(classes, len(classes))

['awe', 'biomorphic_shapes', 'complexity_in_order', 'connection_with_nature', 'mystery', 'natural_materials', 'natural_organisation', 'presence_of_animals', 'presence_of_plants_or_fungi', 'presence_of_water', 'refuge', 'risk', 'unimpeded_views', 'varying_light'] 14


In [6]:
(train_loader, train_data_len) = get_data_loaders(dataset_path, 128, train=True)
(val_loader, test_loader, valid_data_len, test_data_len) = get_data_loaders(dataset_path, 32, train=False)

Dataset ImageFolder
    Number of datapoints: 10373
    Root Location: M:/VieApp/biophilic_training_dataset\train/
    Transforms (if any): Compose(
                             Resize(size=(256, 256), interpolation=PIL.Image.BILINEAR)
                             ToTensor()
                             Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
                         )
    Target Transforms (if any): None
Dataset ImageFolder
    Number of datapoints: 4200
    Root Location: M:/VieApp/biophilic_training_dataset\val/
    Transforms (if any): Compose(
                             Resize(size=(256, 256), interpolation=PIL.Image.BILINEAR)
                             ToTensor()
                             Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
                         )
    Target Transforms (if any): None
Dataset ImageFolder
    Number of datapoints: 4200
    Root Location: M:/VieApp/biophilic_training_dataset\val/
    Transforms (if any)

In [7]:
dataloaders = {
    "train": train_loader,
    "val": val_loader
}
dataset_sizes = {
    "train": train_data_len,
    "val": valid_data_len
}

In [8]:
# now, for the model
device = torch.device('cpu' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [9]:
batch_size= 8
epochs = 1000
lr = 3e-5
gamma = 0.7
seed = 42

In [10]:
import random
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic =True
    
seed_everything(seed)

In [11]:
efficient_transformer = Linformer(
    dim=8,
    seq_len=1025,
    depth=4,
    heads=2,
    k=4
)

model = ViT(
    dim=8,
    image_size=256,
    patch_size=8,
    num_classes=14,
    transformer=efficient_transformer,
    channels=3,).to(device)

In [12]:
#loss function
criterion = nn.CrossEntropyLoss()
#optimizer
optimizer = optim.Adam(model.parameters(), lr=lr)
#scheduler
scheduler = StepLR(optimizer,step_size=1, gamma=gamma)

In [13]:
#training code
for epoch in range(epochs):
    epoch_loss = 0
    epoch_accuracy = 0
    
    for dataset, classes in tqdm(train_loader):
        dataset = dataset.to(device)
        classes = classes.to(device)
        
        output = model(dataset)
        loss = criterion(output, classes)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        acc = (output.argmax(dim=1) == classes).float().mean()
        epoch_accuracy += acc / len(train_loader)
        epoch_loss += loss / len(train_loader)
        
    with torch.no_grad():
        epoch_val_accuracy = 0
        epoch_val_loss = 0
        for dataset, classes in val_loader:
            dataset = dataset.to(device)
            classes = classes.to(device)
            
            val_output = model(dataset)
            val_loss = criterion(val_output, classes)
            
            acc = (val_output.argmax(dim=1) == classes).float().mean()
            epoch_val_accuracy += acc /len(val_loader)
            epoch_val_loss += val_loss / len(val_loader)
            
        print(f"Epoch: {epoch+1} - loss: {epoch_loss:.4f} - acc: {epoch_accuracy:.4f} - val_loss: {epoch_val_loss:.4f} - val_acc: {epoch_val_accuracy:.4f}")

100%|██████████| 82/82 [05:55<00:00,  4.34s/it]


Epoch: 1 - loss: 2.7728 - acc: 0.0736 - val_loss: 2.7519 - val_acc: 0.0807


100%|██████████| 82/82 [04:17<00:00,  3.14s/it]


Epoch: 2 - loss: 2.7345 - acc: 0.0735 - val_loss: 2.7289 - val_acc: 0.0760


100%|██████████| 82/82 [04:31<00:00,  3.31s/it]


...

Epoch: 998 - loss: 2.2144 - acc: 0.2244 - val_loss: 2.7494 - val_acc: 0.1250


100%|██████████| 82/82 [05:50<00:00,  4.28s/it]


Epoch: 999 - loss: 2.2221 - acc: 0.2211 - val_loss: 2.7453 - val_acc: 0.1259


100%|██████████| 82/82 [05:59<00:00,  4.38s/it]


Epoch: 1000 - loss: 2.2168 - acc: 0.2237 - val_loss: 2.7392 - val_acc: 0.1243
